# Scale population for Postcode Sectors and Workplace Zones

In outline:
- get area definitions for Local Authority Districts (LAD - 2011 and 2016), Workplace Zones (WZ), Postcode Sectors (PS) in Great Britain (GB)
- read 2011 census data for workplace population and usual resident population at LAD, WZ and PS
- convert 2011 LAD codes to 2016 equivalents
- read population and employment scenarios and baseline projection at LAD scale
- scale future employment to workplace zones
- scale future population to postcode sectors

In [ ]:
import os
import zipfile

import pandas as pd
import requests
from ukcensusapi.Nomisweb import Nomisweb

## Get boundaries and lookups

In [ ]:
def download(url, filename, dirname=".", force=False):
    if force or not os.path.exists(filename):
        r = requests.get(url, stream=True)
        with open(filename, 'wb') as fd:
            for chunk in r.iter_content(chunk_size=128):
                fd.write(chunk)
    if filename.endswith(".zip"):
        with zipfile.ZipFile(filename,"r") as zf:
            zf.extractall(dirname)    

In [ ]:
# Scotland Workplace Zones
# ref: https://www.nrscotland.gov.uk/statistics-and-data/geography/our-products/census-datasets/2011-census/2011-boundaries
url = "https://www.nrscotland.gov.uk/files//geography/products/workplacezones2011scotland.zip"
download(url, "data/workplacezones2011scotland.zip")

In [ ]:
# England and Wales Workplace Zones
# ref: https://geoportal.statistics.gov.uk/datasets/workplace-zones-december-2011-full-clipped-boundaries-in-england-and-wales
url = "https://opendata.arcgis.com/datasets/a399c2a5922a4beaa080de63c0a218a3_0.zip?outSR=%7B%22latestWkid%22%3A27700%2C%22wkid%22%3A27700%7D"
download(url, "data/workplacezones2011ew.zip", "data")

In [ ]:
# OA to LAD to WZ lookup (EW)
# ref: http://geoportal.statistics.gov.uk/datasets/output-area-to-workplace-zone-to-local-authority-district-december-2011-lookup-in-england-and-wales
url = "https://opendata.arcgis.com/datasets/5a8a6ac972cc4ce4bc02f64f52f8ffd7_0.csv"
download(url, "data/oa-to-workplace-zone-to-lad.csv")

In [ ]:
# GB Postcode Sectors
# ref: https://datashare.is.ed.ac.uk/handle/10283/2597
url = "https://datashare.is.ed.ac.uk/bitstream/handle/10283/2597/GB_Postcodes.zip?sequence=1&isAllowed=y"
download(url, "data/gb_postcodes.zip", "data")

## Get 2011 census data 
- population by Postcode Sector, LAD
- workplace population by Workplace Zone, LAD

England and Wales are covered by Nomis; use UKCensusAPI to access population by postcode sector and LAD.

In [ ]:
api = Nomisweb("cache")

In [ ]:
api.get_metadata("KS101EW")  # Resident population

In [ ]:
coverage_codes = [Nomisweb.GeoCodeLookup["EnglandWales"]]
resolution = 'TYPE276'
geo_codes = api.get_geo_codes(coverage_codes, resolution)
# first and last, split and joined
ew_postcodesector_geo_codes = geo_codes.split(",")[0].split("...")[0] + "..." + geo_codes.split(",")[-1]
ew_postcodesector_geo_codes

In [ ]:
query_params = {}
query_params["date"] = "latest"
query_params["MEASURES"] = "20100"  # Value (as opposed to Percent)
query_params["geography"] = ew_postcodesector_geo_codes # All England and Wales postcodes
query_params["RURAL_URBAN"] = "0"  # Total
query_params["CELL"] = "0"  # All usual residents
query_params["select"] = "GEOGRAPHY_CODE,OBS_VALUE"

pop_postcodesector_ew = api.get_data('KS101EW', query_params)
pop_postcodesector_ew.head(10)

In [ ]:
def unique_geo_len_sum(df):
    return len(df.GEOGRAPHY_CODE.unique()) == len(df), len(df), df.OBS_VALUE.sum()

In [ ]:
unique_geo_len_sum(pop_postcodesector_ew)

In [ ]:
lad_geo_codes = api.get_geo_codes(coverage_codes, 'TYPE464')  # LADs

In [ ]:
query_params = {}
query_params["date"] = "latest"
query_params["MEASURES"] = "20100"  # Value (as opposed to Percent)
query_params["geography"] = lad_geo_codes
query_params["RURAL_URBAN"] = "0"  # Total
query_params["CELL"] = "0"  # All usual residents
query_params["select"] = "GEOGRAPHY_CODE,OBS_VALUE"

pop_lad11_ew = api.get_data('KS101EW', query_params)
pop_lad11_ew.head(10)

In [ ]:
unique_geo_len_sum(pop_lad11_ew)

In [ ]:
api.get_metadata("WP101EW")  # Workplace population

In [ ]:
query_params = {}
query_params["date"] = "latest"
query_params["MEASURES"] = "20100"  # Value (as opposed to Percent)
query_params["geography"] = lad_geo_codes
query_params["select"] = "GEOGRAPHY_CODE,OBS_VALUE"

wp_lad11_ew = api.get_data('WP101EW', query_params)
wp_lad11_ew.head(10)

In [ ]:
unique_geo_len_sum(wp_lad11_ew)

Workplace population is not available by workplace zones through NOMIS, download for England and Wales through INFUSE geo wizard at http://infusecp.mimas.ac.uk/infusewizgeo.aspx

### Citations

England, Northern Ireland, Scotland and Wales 2011 Census
- Office for National Statistics ; National Records of Scotland ; Northern Ireland Statistics and Research Agency (2017): 2011 Census aggregate data. UK Data Service (Edition: February 2017). DOI: http://dx.doi.org/10.5257/census/aggregate-2011-2
This information is licensed under the terms of the Open Government Licence [http://www.nationalarchives.gov.uk/doc/open-government-licence/version/2].

England and Wales 2001 Census
- Office for National Statistics (2011): 2001 Census aggregate data (Edition: May 2011). UK Data Service. DOI: http://dx.doi.org/10.5257/census/aggregate-2001-2


In [ ]:
wp_wz11_ew = pd.read_csv('data/Data_AGE_DAYPOP_ECOACT_UNIT.csv').rename(columns={'GEO_CODE': 'GEOGRAPHY_CODE'})
wp_wz11_ew.head()

In [ ]:
unique_geo_len_sum(wp_wz11_ew)

In [ ]:
# Scottish census tables by postocde sector
#  - zip includes KS101SC.csv Usual Resident Population
url = "https://www.scotlandscensus.gov.uk/ods-web/download/getDownloadFile.html?downloadFileIds=LC%20Postcode%20Sector%20blk"
download(url, "data/postcode-sectors-blk.zip", "data")

In [ ]:
# Scottish Workplace Population by Workplace Zone (and Industry)
url = "http://www.scotlandscensus.gov.uk/documents/additional_tables/WP605SCwz.csv"
download(url, "data/WP605SCwz.csv")